## Create

In [2]:
import re
import pandas as pd
import numpy as np
import os

In [3]:
dir_path = "C:/Users/Daniel/Documents/Git/bank-statement-cleaner/synthetic_data"
savings_csv_path = os.path.join(dir_path, "savings.csv")
chequing_csv_path = os.path.join(dir_path, "chequing.csv")
credit_csv_path = os.path.join(dir_path, "credit.csv")

In [4]:
class CIBCTransactionDescription:
    def __init__(
        self,
        method="",
        type="",
        party=""
    ):
        self.method = method
        self.type = type
        self.party = party

    def to_pd_series(self):
        # convert to lower case to make queries easier
        attributes = [
            self.method,
            self.type,
            self.party
        ]
        for i in range(0, len(attributes)):
            if not attributes[i]:
                attributes[i] = np.nan
            else:
                attributes[i] = attributes[i].lower()
        return pd.Series(attributes)

In [5]:
def parse_debit_description(description):
    tx_type_match = re.search(r'[A-Z][^a-z0-9]*[A-Z]', description) # get transaction type
    if not tx_type_match:
        return CIBCTransactionDescription().to_pd_series()
    tx_type = tx_type_match.group()
    tx_method = description[:tx_type_match.span()[0] - 1]
    for word in [
        "CHARGE",
        "CORRECTION",
        "DEPOSIT",
        "FEE",
        "INTEREST",
        "MEMO",
        "PAY",
        "PURCHASE",
        "TRANSFER",
    ]:
        word_start_index = tx_type.rfind(word) # scan from right
        if word_start_index == -1:
            continue
        tx_type = tx_type[:word_start_index + len(word)]
        # special case for service charges
        if word == "CHARGE":
            return CIBCTransactionDescription(
                method=tx_method,
                type=tx_type
            ).to_pd_series()
        break
    tx_type_end_index = tx_type_match.span()[0] + len(tx_type) - 1
    if tx_type_end_index == len(description) - 1:
        return CIBCTransactionDescription(
            method=tx_method,
            type=tx_type
        ).to_pd_series()
    remainder = description[tx_type_end_index + 2:]
    remainder = remainder.replace("*", "") # delete asterisks
    # find a token with only letters and numbers with at least 1 letter and 1 number
    for token in remainder.split(" "):
        if re.search("^[0-9A-Z]+$", token) and re.search("[0-9]", token) and re.search("[A-Z]", token):
            return CIBCTransactionDescription(
                    method=tx_method,
                    type=tx_type,
                    party=remainder.replace(token, "").strip()
                ).to_pd_series()
    # find a token with only numbers
    token_match = re.search("^[0-9]+(?= )|(?<= )[0-9]+$|(?<= )[0-9]+(?= )", remainder)
    if token_match:
        return CIBCTransactionDescription(
            method=tx_method,
            type=tx_type,
            party=remainder.replace(token_match.group(), "").strip()
        ).to_pd_series()
    return CIBCTransactionDescription(
        method=tx_method,
        type=tx_type
    ).to_pd_series()

In [6]:
def expand_debit(df):
    df[["method", "type", "party"]] = df["description"].apply(parse_debit_description)

In [7]:
def parse_credit_description(description):
    tx_location_match = re.search(r'[^ ]+, .+$', description) # get transaction location
    if not tx_location_match:
        return np.nan
    return description.replace(tx_location_match.group(), "").strip().lower()

In [8]:
def expand_credit(df):
    df["party"] = df["description"].apply(parse_credit_description)

In [9]:
def expand_account_df(account, account_df, expand_fn=None, duplicate_index=True):
    df = account_df.copy()
    date = pd.to_datetime(df["date"])
    df["date"] = date.dt.strftime("%Y-%m-%d")
    df["year"] = date.dt.year
    df["month"] = date.dt.month
    df["day"] = date.dt.day
    df["account"] = account
    df["amount"] = df[["debit", "credit"]].apply(lambda x: x["credit"] if pd.isnull(x["debit"]) else -1 * x["debit"], axis=1)
    df = df.drop(columns=["debit", "credit"])
    if expand_fn:
        expand_fn(df)
    if duplicate_index:
        df["index_copy"] = df.index
    return df

In [10]:
savings_df = pd.read_csv(savings_csv_path, names=["date", "description", "debit", "credit"])
savings_df

,date,description,debit,credit
0,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,NaN,2000.0
1,2025-01-31,Internet Banking E-TRANSFER 497466188861 LANDLORD,1500.0,NaN
2,2025-01-31,Internet Banking E-TRANSFER 023468170802 LANDLORD,283.0,NaN
3,2025-01-31,Internet Banking INTERNET TRANSFER 125959237893,NaN,50000.0
4,2025-02-28,Internet Banking E-TRANSFER 233755062894 LANDLORD,1500.0,NaN
5,2025-02-28,Internet Banking E-TRANSFER 307059499977 LANDLORD,277.0,NaN
6,2025-02-28,Internet Banking INTERNET TRANSFER 302543818917,NaN,50000.0
7,2025-03-31,Internet Banking INTERNET TRANSFER 838173975039,NaN,50000.0
8,2025-03-31,Internet Banking E-TRANSFER 410913019913 LANDLORD,284.0,NaN
9,2025-03-31,Internet Banking E-TRANSFER 528426674492 LANDLORD,1500.0,NaN


In [11]:
expanded_savings_df = expand_account_df("savings", savings_df, expand_fn=expand_debit)
expanded_savings_df

,date,description,year,month,day,account,amount,method,type,party,index_copy
0,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,2025,1,1,savings,2000.0,automated banking machine,atm deposit,NaN,0
1,2025-01-31,Internet Banking E-TRANSFER 497466188861 LANDLORD,2025,1,31,savings,-1500.0,internet banking,e-transfer,landlord,1
2,2025-01-31,Internet Banking E-TRANSFER 023468170802 LANDLORD,2025,1,31,savings,-283.0,internet banking,e-transfer,landlord,2
3,2025-01-31,Internet Banking INTERNET TRANSFER 125959237893,2025,1,31,savings,50000.0,internet banking,internet transfer,NaN,3
4,2025-02-28,Internet Banking E-TRANSFER 233755062894 LANDLORD,2025,2,28,savings,-1500.0,internet banking,e-transfer,landlord,4
5,2025-02-28,Internet Banking E-TRANSFER 307059499977 LANDLORD,2025,2,28,savings,-277.0,internet banking,e-transfer,landlord,5
6,2025-02-28,Internet Banking INTERNET TRANSFER 302543818917,2025,2,28,savings,50000.0,internet banking,internet transfer,NaN,6
7,2025-03-31,Internet Banking INTERNET TRANSFER 838173975039,2025,3,31,savings,50000.0,internet banking,internet transfer,NaN,7
8,2025-03-31,Internet Banking E-TRANSFER 410913019913 LANDLORD,2025,3,31,savings,-284.0,internet banking,e-transfer,landlord,8
9,2025-03-31,Internet Banking E-TRANSFER 528426674492 LANDLORD,2025,3,31,savings,-1500.0,internet banking,e-transfer,landlord,9


In [12]:
chequing_df = pd.read_csv(chequing_csv_path, names=["date", "description", "debit", "credit"])
chequing_df

,date,description,debit,credit
0,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,NaN,2000.0
1,2025-01-01,Internet Banking E-TRANSFER 198387442408 BLUE ...,50.0,NaN
2,2025-01-02,Point of Sale - Interac RETAIL PURCHASE 739855...,16.0,NaN
3,2025-01-02,Point of Sale - Visa Debit VISA DEBIT RETAIL P...,3.0,NaN
4,2025-01-03,Point of Sale - Visa Debit VISA DEBIT RETAIL P...,10.0,NaN
...,...,...,...,...
416,2025-12-01,Internet Banking E-TRANSFER 510912177359 BLUE ...,50.0,NaN
417,2025-12-02,Internet Banking INTERNET TRANSFER 872872991979,54.0,NaN
418,2025-12-02,Point of Sale - Interac RETAIL PURCHASE 925576...,16.0,NaN
419,2025-12-31,Internet Banking INTERNET TRANSFER 794179128749,50000.0,NaN


In [13]:
expanded_chequing_df = expand_account_df("chequing", chequing_df, expand_fn=expand_debit)
expanded_chequing_df

,date,description,year,month,day,account,amount,method,type,party,index_copy
0,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,2025,1,1,chequing,2000.0,automated banking machine,atm deposit,NaN,0
1,2025-01-01,Internet Banking E-TRANSFER 198387442408 BLUE ...,2025,1,1,chequing,-50.0,internet banking,e-transfer,blue hockey club,1
2,2025-01-02,Point of Sale - Interac RETAIL PURCHASE 739855...,2025,1,2,chequing,-16.0,point of sale - interac,retail purchase,pizza,2
3,2025-01-02,Point of Sale - Visa Debit VISA DEBIT RETAIL P...,2025,1,2,chequing,-3.0,point of sale - visa debit,visa debit retail purchase,presto,3
4,2025-01-03,Point of Sale - Visa Debit VISA DEBIT RETAIL P...,2025,1,3,chequing,-10.0,point of sale - visa debit,visa debit retail purchase,presto,4
...,...,...,...,...,...,...,...,...,...,...,...
416,2025-12-01,Internet Banking E-TRANSFER 510912177359 BLUE ...,2025,12,1,chequing,-50.0,internet banking,e-transfer,blue hockey club,416
417,2025-12-02,Internet Banking INTERNET TRANSFER 872872991979,2025,12,2,chequing,-54.0,internet banking,internet transfer,NaN,417
418,2025-12-02,Point of Sale - Interac RETAIL PURCHASE 925576...,2025,12,2,chequing,-16.0,point of sale - interac,retail purchase,pizza,418
419,2025-12-31,Internet Banking INTERNET TRANSFER 794179128749,2025,12,31,chequing,-50000.0,internet banking,internet transfer,NaN,419


In [14]:
def load_cibc_credit_csv(csv_path):
    credit_df = pd.read_csv(csv_path, usecols=range(0, 4), names=["date", "description", "debit", "credit"])
    return credit_df

In [15]:
credit_df = load_cibc_credit_csv(credit_csv_path)
credit_df

,date,description,debit,credit
0,2025-01-07,"WALMART TORONTO, ON",78.0,NaN
1,2025-01-07,PAYMENT THANK YOU/PAIEMEN T MERCI,NaN,78.0
2,2025-01-14,"WALMART TORONTO, ON",65.0,NaN
3,2025-01-14,PAYMENT THANK YOU/PAIEMEN T MERCI,NaN,65.0
4,2025-01-21,"WALMART TORONTO, ON",92.0,NaN
...,...,...,...,...
91,2025-11-18,PAYMENT THANK YOU/PAIEMEN T MERCI,NaN,60.0
92,2025-11-25,"WALMART TORONTO, ON",83.0,NaN
93,2025-11-25,PAYMENT THANK YOU/PAIEMEN T MERCI,NaN,83.0
94,2025-12-02,"WALMART TORONTO, ON",54.0,NaN


In [16]:
expanded_credit_df = expand_account_df(
    "credit",
    credit_df,
    expand_fn=expand_credit,
    duplicate_index=False
)
expanded_credit_df

,date,description,year,month,day,account,amount,party
0,2025-01-07,"WALMART TORONTO, ON",2025,1,7,credit,-78.0,walmart
1,2025-01-07,PAYMENT THANK YOU/PAIEMEN T MERCI,2025,1,7,credit,78.0,NaN
2,2025-01-14,"WALMART TORONTO, ON",2025,1,14,credit,-65.0,walmart
3,2025-01-14,PAYMENT THANK YOU/PAIEMEN T MERCI,2025,1,14,credit,65.0,NaN
4,2025-01-21,"WALMART TORONTO, ON",2025,1,21,credit,-92.0,walmart
...,...,...,...,...,...,...,...,...
91,2025-11-18,PAYMENT THANK YOU/PAIEMEN T MERCI,2025,11,18,credit,60.0,NaN
92,2025-11-25,"WALMART TORONTO, ON",2025,11,25,credit,-83.0,walmart
93,2025-11-25,PAYMENT THANK YOU/PAIEMEN T MERCI,2025,11,25,credit,83.0,NaN
94,2025-12-02,"WALMART TORONTO, ON",2025,12,2,credit,-54.0,walmart


In [17]:
def create_indexed_df(dataframes):
    df = pd.concat(dataframes).sort_values(by=["date"])
    df["uid"] = df.reset_index(drop=True).index + 1
    return df

In [18]:
combined_expanded_df = create_indexed_df([
    expanded_savings_df,
    expanded_chequing_df,
    expanded_credit_df
])
combined_expanded_df

,date,description,year,month,day,account,amount,method,type,party,index_copy,uid
0,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,2025,1,1,savings,2000.0,automated banking machine,atm deposit,NaN,0.0,1
1,2025-01-01,Internet Banking E-TRANSFER 198387442408 BLUE ...,2025,1,1,chequing,-50.0,internet banking,e-transfer,blue hockey club,1.0,2
0,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,2025,1,1,chequing,2000.0,automated banking machine,atm deposit,NaN,0.0,3
3,2025-01-02,Point of Sale - Visa Debit VISA DEBIT RETAIL P...,2025,1,2,chequing,-3.0,point of sale - visa debit,visa debit retail purchase,presto,3.0,4
2,2025-01-02,Point of Sale - Interac RETAIL PURCHASE 739855...,2025,1,2,chequing,-16.0,point of sale - interac,retail purchase,pizza,2.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
419,2025-12-31,Internet Banking INTERNET TRANSFER 794179128749,2025,12,31,chequing,-50000.0,internet banking,internet transfer,NaN,419.0,550
420,2025-12-31,Branch Transaction CREDIT MEMO,2025,12,31,chequing,7000.0,branch transaction,credit memo,NaN,420.0,551
36,2025-12-31,Internet Banking INTERNET TRANSFER 794179128749,2025,12,31,savings,50000.0,internet banking,internet transfer,NaN,36.0,552
35,2025-12-31,Internet Banking E-TRANSFER 181227351012 LANDLORD,2025,12,31,savings,-1500.0,internet banking,e-transfer,landlord,35.0,553


In [19]:
expanded_savings_df = expanded_savings_df.sort_index()
expanded_savings_df["uid"] = combined_expanded_df.loc[combined_expanded_df["account"] == "savings"].sort_index()["uid"]
expanded_savings_df

,date,description,year,month,day,account,amount,method,type,party,index_copy,uid
0,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,2025,1,1,savings,2000.0,automated banking machine,atm deposit,NaN,0,1
1,2025-01-31,Internet Banking E-TRANSFER 497466188861 LANDLORD,2025,1,31,savings,-1500.0,internet banking,e-transfer,landlord,1,46
2,2025-01-31,Internet Banking E-TRANSFER 023468170802 LANDLORD,2025,1,31,savings,-283.0,internet banking,e-transfer,landlord,2,45
3,2025-01-31,Internet Banking INTERNET TRANSFER 125959237893,2025,1,31,savings,50000.0,internet banking,internet transfer,NaN,3,51
4,2025-02-28,Internet Banking E-TRANSFER 233755062894 LANDLORD,2025,2,28,savings,-1500.0,internet banking,e-transfer,landlord,4,93
5,2025-02-28,Internet Banking E-TRANSFER 307059499977 LANDLORD,2025,2,28,savings,-277.0,internet banking,e-transfer,landlord,5,94
6,2025-02-28,Internet Banking INTERNET TRANSFER 302543818917,2025,2,28,savings,50000.0,internet banking,internet transfer,NaN,6,96
7,2025-03-31,Internet Banking INTERNET TRANSFER 838173975039,2025,3,31,savings,50000.0,internet banking,internet transfer,NaN,7,145
8,2025-03-31,Internet Banking E-TRANSFER 410913019913 LANDLORD,2025,3,31,savings,-284.0,internet banking,e-transfer,landlord,8,143
9,2025-03-31,Internet Banking E-TRANSFER 528426674492 LANDLORD,2025,3,31,savings,-1500.0,internet banking,e-transfer,landlord,9,144


In [20]:
expanded_chequing_df = expanded_chequing_df.sort_index()
expanded_chequing_df["uid"] = combined_expanded_df.loc[combined_expanded_df["account"] == "chequing"].sort_index()["uid"]
expanded_chequing_df

,date,description,year,month,day,account,amount,method,type,party,index_copy,uid
0,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,2025,1,1,chequing,2000.0,automated banking machine,atm deposit,NaN,0,3
1,2025-01-01,Internet Banking E-TRANSFER 198387442408 BLUE ...,2025,1,1,chequing,-50.0,internet banking,e-transfer,blue hockey club,1,2
2,2025-01-02,Point of Sale - Interac RETAIL PURCHASE 739855...,2025,1,2,chequing,-16.0,point of sale - interac,retail purchase,pizza,2,5
3,2025-01-02,Point of Sale - Visa Debit VISA DEBIT RETAIL P...,2025,1,2,chequing,-3.0,point of sale - visa debit,visa debit retail purchase,presto,3,4
4,2025-01-03,Point of Sale - Visa Debit VISA DEBIT RETAIL P...,2025,1,3,chequing,-10.0,point of sale - visa debit,visa debit retail purchase,presto,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...
416,2025-12-01,Internet Banking E-TRANSFER 510912177359 BLUE ...,2025,12,1,chequing,-50.0,internet banking,e-transfer,blue hockey club,416,545
417,2025-12-02,Internet Banking INTERNET TRANSFER 872872991979,2025,12,2,chequing,-54.0,internet banking,internet transfer,NaN,417,546
418,2025-12-02,Point of Sale - Interac RETAIL PURCHASE 925576...,2025,12,2,chequing,-16.0,point of sale - interac,retail purchase,pizza,418,547
419,2025-12-31,Internet Banking INTERNET TRANSFER 794179128749,2025,12,31,chequing,-50000.0,internet banking,internet transfer,NaN,419,550


In [21]:
expanded_credit_df = expanded_credit_df.sort_index()
expanded_credit_df["uid"] = combined_expanded_df.loc[combined_expanded_df["account"] == "credit"].sort_index()["uid"]
expanded_credit_df

,date,description,year,month,day,account,amount,party,uid
0,2025-01-07,"WALMART TORONTO, ON",2025,1,7,credit,-78.0,walmart,13
1,2025-01-07,PAYMENT THANK YOU/PAIEMEN T MERCI,2025,1,7,credit,78.0,NaN,10
2,2025-01-14,"WALMART TORONTO, ON",2025,1,14,credit,-65.0,walmart,23
3,2025-01-14,PAYMENT THANK YOU/PAIEMEN T MERCI,2025,1,14,credit,65.0,NaN,22
4,2025-01-21,"WALMART TORONTO, ON",2025,1,21,credit,-92.0,walmart,32
...,...,...,...,...,...,...,...,...,...
91,2025-11-18,PAYMENT THANK YOU/PAIEMEN T MERCI,2025,11,18,credit,60.0,NaN,520
92,2025-11-25,"WALMART TORONTO, ON",2025,11,25,credit,-83.0,walmart,532
93,2025-11-25,PAYMENT THANK YOU/PAIEMEN T MERCI,2025,11,25,credit,83.0,NaN,533
94,2025-12-02,"WALMART TORONTO, ON",2025,12,2,credit,-54.0,walmart,548


In [22]:
merged_df = expanded_savings_df.merge(expanded_chequing_df, left_on="description", right_on="description")
merged_df = merged_df.loc[merged_df["amount_x"] == -1 * merged_df["amount_y"]]
merged_df

,date_x,description,year_x,month_x,day_x,account_x,amount_x,method_x,type_x,party_x,...,year_y,month_y,day_y,account_y,amount_y,method_y,type_y,party_y,index_copy_y,uid_y
1,2025-01-31,Internet Banking INTERNET TRANSFER 125959237893,2025,1,31,savings,50000.0,internet banking,internet transfer,NaN,...,2025,1,31,chequing,-50000.0,internet banking,internet transfer,NaN,38,50
2,2025-02-28,Internet Banking INTERNET TRANSFER 302543818917,2025,2,28,savings,50000.0,internet banking,internet transfer,NaN,...,2025,2,28,chequing,-50000.0,internet banking,internet transfer,NaN,72,95
3,2025-03-31,Internet Banking INTERNET TRANSFER 838173975039,2025,3,31,savings,50000.0,internet banking,internet transfer,NaN,...,2025,3,31,chequing,-50000.0,internet banking,internet transfer,NaN,110,142
4,2025-04-30,Internet Banking INTERNET TRANSFER 732015329788,2025,4,30,savings,50000.0,internet banking,internet transfer,NaN,...,2025,4,30,chequing,-50000.0,internet banking,internet transfer,NaN,147,195
5,2025-05-31,Internet Banking INTERNET TRANSFER 199266785895,2025,5,31,savings,50000.0,internet banking,internet transfer,NaN,...,2025,5,31,chequing,-50000.0,internet banking,internet transfer,NaN,184,243
6,2025-06-30,Internet Banking INTERNET TRANSFER 647328542519,2025,6,30,savings,50000.0,internet banking,internet transfer,NaN,...,2025,6,30,chequing,-50000.0,internet banking,internet transfer,NaN,221,287
7,2025-07-31,Internet Banking INTERNET TRANSFER 224938125884,2025,7,31,savings,50000.0,internet banking,internet transfer,NaN,...,2025,7,31,chequing,-50000.0,internet banking,internet transfer,NaN,261,343
8,2025-08-31,Internet Banking INTERNET TRANSFER 213841671997,2025,8,31,savings,50000.0,internet banking,internet transfer,NaN,...,2025,8,31,chequing,-50000.0,internet banking,internet transfer,NaN,301,392
9,2025-09-30,Internet Banking INTERNET TRANSFER 055336254384,2025,9,30,savings,50000.0,internet banking,internet transfer,NaN,...,2025,9,30,chequing,-50000.0,internet banking,internet transfer,NaN,337,439
10,2025-10-31,Internet Banking INTERNET TRANSFER 576395623352,2025,10,31,savings,50000.0,internet banking,internet transfer,NaN,...,2025,10,31,chequing,-50000.0,internet banking,internet transfer,NaN,376,492


In [23]:
internal_transfer_df = pd.concat([
    expanded_savings_df.loc[merged_df["index_copy_x"]],
    expanded_chequing_df.loc[merged_df["index_copy_y"]]
]).drop(columns=["index_copy", "party"]).sort_values(by=["uid"]).reset_index(drop=True)
internal_transfer_df

,date,description,year,month,day,account,amount,method,type,uid
0,2025-01-31,Internet Banking INTERNET TRANSFER 125959237893,2025,1,31,chequing,-50000.0,internet banking,internet transfer,50
1,2025-01-31,Internet Banking INTERNET TRANSFER 125959237893,2025,1,31,savings,50000.0,internet banking,internet transfer,51
2,2025-02-28,Internet Banking INTERNET TRANSFER 302543818917,2025,2,28,chequing,-50000.0,internet banking,internet transfer,95
3,2025-02-28,Internet Banking INTERNET TRANSFER 302543818917,2025,2,28,savings,50000.0,internet banking,internet transfer,96
4,2025-03-31,Internet Banking INTERNET TRANSFER 838173975039,2025,3,31,chequing,-50000.0,internet banking,internet transfer,142
5,2025-03-31,Internet Banking INTERNET TRANSFER 838173975039,2025,3,31,savings,50000.0,internet banking,internet transfer,145
6,2025-04-30,Internet Banking INTERNET TRANSFER 732015329788,2025,4,30,savings,50000.0,internet banking,internet transfer,192
7,2025-04-30,Internet Banking INTERNET TRANSFER 732015329788,2025,4,30,chequing,-50000.0,internet banking,internet transfer,195
8,2025-05-31,Internet Banking INTERNET TRANSFER 199266785895,2025,5,31,savings,50000.0,internet banking,internet transfer,241
9,2025-05-31,Internet Banking INTERNET TRANSFER 199266785895,2025,5,31,chequing,-50000.0,internet banking,internet transfer,243


In [24]:
raw_debit_df = pd.concat([
    expanded_savings_df.drop(merged_df["index_copy_x"]),
    expanded_chequing_df.drop(merged_df["index_copy_y"])
]).drop(columns=["index_copy"]).reset_index(drop=True)
raw_debit_df

,date,description,year,month,day,account,amount,method,type,party,uid
0,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,2025,1,1,savings,2000.0,automated banking machine,atm deposit,NaN,1
1,2025-01-31,Internet Banking E-TRANSFER 497466188861 LANDLORD,2025,1,31,savings,-1500.0,internet banking,e-transfer,landlord,46
2,2025-01-31,Internet Banking E-TRANSFER 023468170802 LANDLORD,2025,1,31,savings,-283.0,internet banking,e-transfer,landlord,45
3,2025-02-28,Internet Banking E-TRANSFER 233755062894 LANDLORD,2025,2,28,savings,-1500.0,internet banking,e-transfer,landlord,93
4,2025-02-28,Internet Banking E-TRANSFER 307059499977 LANDLORD,2025,2,28,savings,-277.0,internet banking,e-transfer,landlord,94
...,...,...,...,...,...,...,...,...,...,...,...
429,2025-11-30,Point of Sale - Visa Debit VISA DEBIT RETAIL P...,2025,11,30,chequing,-3.0,point of sale - visa debit,visa debit retail purchase,presto,542
430,2025-12-01,Internet Banking E-TRANSFER 510912177359 BLUE ...,2025,12,1,chequing,-50.0,internet banking,e-transfer,blue hockey club,545
431,2025-12-02,Internet Banking INTERNET TRANSFER 872872991979,2025,12,2,chequing,-54.0,internet banking,internet transfer,NaN,546
432,2025-12-02,Point of Sale - Interac RETAIL PURCHASE 925576...,2025,12,2,chequing,-16.0,point of sale - interac,retail purchase,pizza,547


In [25]:
internal_payment_from_debit_df = raw_debit_df.loc[
    (raw_debit_df["method"] == "internet banking") &
    (raw_debit_df["type"] == "internet transfer")
].drop(columns=["party"])
internal_payment_from_debit_df

,date,description,year,month,day,account,amount,method,type,uid
33,2025-01-07,Internet Banking INTERNET TRANSFER 116494411368,2025,1,7,chequing,-78.0,internet banking,internet transfer,12
41,2025-01-14,Internet Banking INTERNET TRANSFER 493121374759,2025,1,14,chequing,-65.0,internet banking,internet transfer,21
50,2025-01-21,Internet Banking INTERNET TRANSFER 040555104405,2025,1,21,chequing,-92.0,internet banking,internet transfer,30
58,2025-01-28,Internet Banking INTERNET TRANSFER 869401334317,2025,1,28,chequing,-75.0,internet banking,internet transfer,41
68,2025-02-04,Internet Banking INTERNET TRANSFER 286836600491,2025,2,4,chequing,-86.0,internet banking,internet transfer,59
75,2025-02-11,Internet Banking INTERNET TRANSFER 039959504015,2025,2,11,chequing,-86.0,internet banking,internet transfer,67
83,2025-02-18,Internet Banking INTERNET TRANSFER 379970909730,2025,2,18,chequing,-82.0,internet banking,internet transfer,79
91,2025-02-25,Internet Banking INTERNET TRANSFER 607465394041,2025,2,25,chequing,-94.0,internet banking,internet transfer,88
101,2025-03-04,Internet Banking INTERNET TRANSFER 681876139712,2025,3,4,chequing,-55.0,internet banking,internet transfer,104
110,2025-03-11,Internet Banking INTERNET TRANSFER 607294766562,2025,3,11,chequing,-56.0,internet banking,internet transfer,113


In [26]:
credit_payment_bool_series = expanded_credit_df["description"].str.contains("PAYMENT THANK YOU")

In [27]:
internal_payment_df = pd.concat([
    internal_payment_from_debit_df,
    expanded_credit_df.loc[credit_payment_bool_series]
]).drop(columns=["party"]).sort_values(by=["uid"]).reset_index(drop=True)
internal_payment_df

,date,description,year,month,day,account,amount,method,type,uid
0,2025-01-07,PAYMENT THANK YOU/PAIEMEN T MERCI,2025,1,7,credit,78.0,NaN,NaN,10
1,2025-01-07,Internet Banking INTERNET TRANSFER 116494411368,2025,1,7,chequing,-78.0,internet banking,internet transfer,12
2,2025-01-14,Internet Banking INTERNET TRANSFER 493121374759,2025,1,14,chequing,-65.0,internet banking,internet transfer,21
3,2025-01-14,PAYMENT THANK YOU/PAIEMEN T MERCI,2025,1,14,credit,65.0,NaN,NaN,22
4,2025-01-21,Internet Banking INTERNET TRANSFER 040555104405,2025,1,21,chequing,-92.0,internet banking,internet transfer,30
...,...,...,...,...,...,...,...,...,...,...
91,2025-11-18,Internet Banking INTERNET TRANSFER 795273423770,2025,11,18,chequing,-60.0,internet banking,internet transfer,522
92,2025-11-25,Internet Banking INTERNET TRANSFER 803749812307,2025,11,25,chequing,-83.0,internet banking,internet transfer,531
93,2025-11-25,PAYMENT THANK YOU/PAIEMEN T MERCI,2025,11,25,credit,83.0,NaN,NaN,533
94,2025-12-02,Internet Banking INTERNET TRANSFER 872872991979,2025,12,2,chequing,-54.0,internet banking,internet transfer,546


In [28]:
debit_df = raw_debit_df.drop(internal_payment_from_debit_df.index).reset_index(drop=True)
debit_df

,date,description,year,month,day,account,amount,method,type,party,uid
0,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,2025,1,1,savings,2000.0,automated banking machine,atm deposit,NaN,1
1,2025-01-31,Internet Banking E-TRANSFER 497466188861 LANDLORD,2025,1,31,savings,-1500.0,internet banking,e-transfer,landlord,46
2,2025-01-31,Internet Banking E-TRANSFER 023468170802 LANDLORD,2025,1,31,savings,-283.0,internet banking,e-transfer,landlord,45
3,2025-02-28,Internet Banking E-TRANSFER 233755062894 LANDLORD,2025,2,28,savings,-1500.0,internet banking,e-transfer,landlord,93
4,2025-02-28,Internet Banking E-TRANSFER 307059499977 LANDLORD,2025,2,28,savings,-277.0,internet banking,e-transfer,landlord,94
...,...,...,...,...,...,...,...,...,...,...,...
381,2025-11-30,Branch Transaction CREDIT MEMO,2025,11,30,chequing,7000.0,branch transaction,credit memo,NaN,540
382,2025-11-30,Point of Sale - Visa Debit VISA DEBIT RETAIL P...,2025,11,30,chequing,-3.0,point of sale - visa debit,visa debit retail purchase,presto,542
383,2025-12-01,Internet Banking E-TRANSFER 510912177359 BLUE ...,2025,12,1,chequing,-50.0,internet banking,e-transfer,blue hockey club,545
384,2025-12-02,Point of Sale - Interac RETAIL PURCHASE 925576...,2025,12,2,chequing,-16.0,point of sale - interac,retail purchase,pizza,547


In [29]:
def get_sign(amount):
    if amount > 0:
        return "income"
    elif amount < 0:
        return "expense"
    return "zero-value"

In [30]:
cash_flow_df = pd.concat([
    debit_df,
    expanded_credit_df.loc[~credit_payment_bool_series]
]).sort_values(by=["uid"]).reset_index(drop=True)
cash_flow_df["sign"] = cash_flow_df["amount"].apply(get_sign)
cash_flow_df

,date,description,year,month,day,account,amount,method,type,party,uid,sign
0,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,2025,1,1,savings,2000.0,automated banking machine,atm deposit,NaN,1,income
1,2025-01-01,Internet Banking E-TRANSFER 198387442408 BLUE ...,2025,1,1,chequing,-50.0,internet banking,e-transfer,blue hockey club,2,expense
2,2025-01-01,Automated Banking Machine ATM DEPOSIT TORONTO,2025,1,1,chequing,2000.0,automated banking machine,atm deposit,NaN,3,income
3,2025-01-02,Point of Sale - Visa Debit VISA DEBIT RETAIL P...,2025,1,2,chequing,-3.0,point of sale - visa debit,visa debit retail purchase,presto,4,expense
4,2025-01-02,Point of Sale - Interac RETAIL PURCHASE 739855...,2025,1,2,chequing,-16.0,point of sale - interac,retail purchase,pizza,5,expense
...,...,...,...,...,...,...,...,...,...,...,...,...
429,2025-12-02,Point of Sale - Interac RETAIL PURCHASE 925576...,2025,12,2,chequing,-16.0,point of sale - interac,retail purchase,pizza,547,expense
430,2025-12-02,"WALMART TORONTO, ON",2025,12,2,credit,-54.0,NaN,NaN,walmart,548,expense
431,2025-12-31,Branch Transaction CREDIT MEMO,2025,12,31,chequing,7000.0,branch transaction,credit memo,NaN,551,income
432,2025-12-31,Internet Banking E-TRANSFER 181227351012 LANDLORD,2025,12,31,savings,-1500.0,internet banking,e-transfer,landlord,553,expense


In [31]:
with pd.ExcelWriter('output.xlsx') as writer:
    cash_flow_df.to_excel(writer, sheet_name="cash_flow", index=False)
    internal_transfer_df.to_excel(writer, sheet_name="internal_transfer", index=False)
    internal_payment_df.to_excel(writer, sheet_name="internal_payment", index=False)

## Command Test

In [33]:
# db_df = pd.read_excel("C:/Users/Daniel/Documents/Git/bank-statement-cleaner/outdated.xlsx", sheet_name=None)
# db_df

In [34]:
# internal_payment_df.loc[~internal_payment_df["uid"].isin(db_df["internal_payment"]["uid"])]